<a href="https://colab.research.google.com/github/xvanbeurden/Deep_learning/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://data.ciirc.cvut.cz/public/projects/2015netVLAD/Pittsburgh250k/000.tar
!wget https://data.ciirc.cvut.cz/public/projects/2015netVLAD/Pittsburgh250k/001.tar
!wget https://data.ciirc.cvut.cz/public/projects/2015netVLAD/Pittsburgh250k/002.tar
!wget https://data.ciirc.cvut.cz/public/projects/2015netVLAD/Pittsburgh250k/003.tar
!wget https://data.ciirc.cvut.cz/public/projects/2015netVLAD/Pittsburgh250k/004.tar
!wget https://data.ciirc.cvut.cz/public/projects/2015netVLAD/Pittsburgh250k/005.tar
!wget https://data.ciirc.cvut.cz/public/projects/2015netVLAD/Pittsburgh250k/006.tar
!wget https://data.ciirc.cvut.cz/public/projects/2015netVLAD/Pittsburgh250k/007.tar
!wget https://data.ciirc.cvut.cz/public/projects/2015netVLAD/Pittsburgh250k/008.tar
!wget https://data.ciirc.cvut.cz/public/projects/2015netVLAD/Pittsburgh250k/009.tar
!wget https://data.ciirc.cvut.cz/public/projects/2015netVLAD/Pittsburgh250k/010.tar
!wget https://data.ciirc.cvut.cz/public/projects/2015netVLAD/Pittsburgh250k/queries_real.tar

--2021-04-12 11:51:10--  https://data.ciirc.cvut.cz/public/projects/2015netVLAD/Pittsburgh250k/000.tar
Resolving data.ciirc.cvut.cz (data.ciirc.cvut.cz)... 147.32.71.18
Connecting to data.ciirc.cvut.cz (data.ciirc.cvut.cz)|147.32.71.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 922941440 (880M) [application/x-tar]
Saving to: ‘000.tar’

000.tar             100%[===================>] 880.19M  29.7MB/s    in 31s     

2021-04-12 11:51:41 (28.7 MB/s) - ‘000.tar’ saved [922941440/922941440]

--2021-04-12 11:51:41--  https://data.ciirc.cvut.cz/public/projects/2015netVLAD/Pittsburgh250k/001.tar
Resolving data.ciirc.cvut.cz (data.ciirc.cvut.cz)... 147.32.71.18
Connecting to data.ciirc.cvut.cz (data.ciirc.cvut.cz)|147.32.71.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 854702080 (815M) [application/x-tar]
Saving to: ‘001.tar’

001.tar             100%[===================>] 815.11M  30.9MB/s    in 27s     

2021-04-12 11:52:09 (30.0 M

In [ ]:
!tar -xvf 000.tar
!tar -xvf 001.tar
!tar -xvf 002.tar
!tar -xvf 003.tar
!tar -xvf 004.tar
!tar -xvf 005.tar
!tar -xvf 006.tar
!tar -xvf 007.tar
!tar -xvf 008.tar
!tar -xvf 009.tar
!tar -xvf 010.tar
!tar -xvf queries_real.tar

Streaming output truncated to the last 5000 lines.
queries_real/004/004039_pitch2_yaw4.jpg
queries_real/004/004884_pitch2_yaw3.jpg
queries_real/004/004137_pitch1_yaw8.jpg
queries_real/004/004362_pitch1_yaw2.jpg
queries_real/004/004828_pitch1_yaw5.jpg
queries_real/004/004211_pitch1_yaw5.jpg
queries_real/004/004583_pitch1_yaw1.jpg
queries_real/004/004309_pitch2_yaw1.jpg
queries_real/004/004444_pitch2_yaw11.jpg
queries_real/004/004725_pitch1_yaw6.jpg
queries_real/004/004386_pitch2_yaw6.jpg
queries_real/004/004837_pitch2_yaw6.jpg
queries_real/004/004576_pitch2_yaw5.jpg
queries_real/004/004522_pitch2_yaw8.jpg
queries_real/004/004169_pitch2_yaw1.jpg
queries_real/004/004112_pitch1_yaw8.jpg
queries_real/004/004834_pitch2_yaw4.jpg
queries_real/004/004002_pitch1_yaw5.jpg
queries_real/004/004671_pitch1_yaw1.jpg
queries_real/004/004751_pitch2_yaw11.jpg
queries_real/004/004302_pitch2_yaw11.jpg
queries_real/004/004039_pitch1_yaw6.jpg
queries_real/004/004173_pitch1_yaw3.jpg
queries_real/004/004634_pi

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
!pip install tensorboardX
!pip install faiss-gpu

     |████████████████████████████████| 122kB 14.4MB/s 
     |████████████████████████████████| 89.4MB 36kB/s 


In [ ]:
from __future__ import print_function
import argparse
from math import log10, ceil
import random, shutil, json
from os.path import join, exists, isfile, realpath, dirname
from os import makedirs, remove, chdir, environ

from scipy.io import loadmat
from random import randint, random
from collections import namedtuple
from collections import OrderedDict

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch.utils.data.dataset import Subset
import torchvision.transforms as transforms
from PIL import Image
from datetime import datetime
import torchvision.datasets as datasets
import torchvision.models as models
import h5py
import tensorflow as tf
import faiss
#import cache_magic

#import faiss
# %load_ext tensorboard
from tensorboardX import SummaryWriter
import numpy as np
import time
# import YourModel
# import Dataloader
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

config = OrderedDict(
    # ==== MODE ====
    mode = 'train', #train/test/cluster  #Cluster once!

    # === DATASET ===
    train_size=1000,
    validation_size=500,
    test_size=500,

    # === OPTIMIZER ===
    CNN= 'VGG', #or AlexNet
    batch_size = 4 ,
    epochs=1,
    num_workers=2,
    optimizer= 'SGD',
    learning_rate=0.008,
    NetVLAD = True, #otherwise maxpooling
    cache_refresh_rate = 1000,
    clusters = 64,
    momentum = 0.9,
    weightdecay = 0.001,
    lrGamma = 0.5,
    lrStep = 5,
    myPath = r'content/gdrive/MyDrive/Colab_Notebooks/Main_folder', #save data to this folder!!!!!!!!!
    margin = 0.1,
    #####=== More ===
    cuda = True,
    patience_earlystopping = 3
    )

#this normailization you need if you use NetVLAD. Used in main... 
class L2Norm(nn.Module):
    def __init__(self, dim=1):
        super().__init__()
        self.dim = dim

    def forward(self, input):
        return F.normalize(input, p=2, dim=self.dim)

class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

from sklearn.cluster import KMeans
def centroids(cluster_set):
  
  total_n_descr = 50000
  descr_per_image = 100
  n_im = 500

  if config['CNN'].lower() == 'alexnet':
   encoder_dim = 256

  if config['CNN'].lower() == 'vgg':
   encoder_dim = 512
  # randomsampler =  torch.utils.data.RandomSampler(np.random.choice(len(cluster_set), replacement=False, num_samples=n_im))
  randomsampler =  torch.utils.data.RandomSampler(np.random.choice(len(cluster_set), n_im, replace=False)) # vgm is dit correct nu
  cluster_data_loader = DataLoader(dataset=cluster_set, 
                num_workers=config['num_workers'], batch_size=config['batch_size'], shuffle=False, 
                pin_memory=config['cuda'],
                sampler=randomsampler)
  
  #if not exists(join(config['myPath'], 'centroids')):
  #  makedirs(join(config['myPath'], 'centroids'))

  #initcache = join(config['myPath'], 'centroids', config['mode'] + '_' + cluster_set.dataset + '_' + str(config['clusters']) + '_desc_cen.hdf5')
  initcache =  'centroids_trainingset'     #join(config['myPath'], 'centroids', config['mode'] + '_' + 'training_dataset_' + '_' + str(config['clusters']) + '_desc_cen.h5py')
  #remove(initcache)
  # initcache =  'centroids_trainingset'     #join(config['myPath'], 'centroids', config['mode'] + '_' + 'training_dataset_' + '_' + str(config['clusters']) + '_desc_cen.h5py')
  # model = nn.Module() 
  
  with h5py.File(initcache, mode='w') as h5: 
      with torch.no_grad():
        model.eval()
        print('====> Extracting Descriptors')
        dbFeat = h5.create_dataset("descriptors", 
                    [total_n_descr, encoder_dim], 
                    dtype=np.float32)

        for iteration, (input, indices) in enumerate(cluster_data_loader, 1):
          try:
            input = input.to(device)
            NetVLAD = False
            if CNN == VGG:
              descriptors = VGG(input) #TODO: this is a little different for NetVLAD of course
            if CNN == AlexNet:
              descriptors = AlexNet(input) #TODO: this is a little different for NetVLAD of course

            batchindex = (iteration-1)*config['batch_size']*descr_per_image
            for i in range(descriptors.size(0)):
                sample = np.random.choice(descriptors.size(1), 100, replace=False)
                startix = batchindex + i*nPerImage
                dbFeat[startix:startix+nPerImage, :] = image_descriptors[ix, sample, :].detach().cpu().numpy()
          except:
            pass
        # del input, descriptors
        
        print('====> Clustering..')
        niter = 100
        # kmeans = KMeans(encoder_dim, config['clusters'], max_iter=niter, verbose=0)
        kmeans = KMeans(config['clusters'], init = 'k-means++', max_iter=niter, verbose=0, n_jobs=-1)
        kmeans.fit(dbFeat[...])
        
        h5.create_dataset('centroids', data=kmeans.cluster_centers_)

# def train_epoch(dataloader, model, criterion, optimizer, epoch):
def train_epoch(epoch, train_dataset, query_train_set, model):
    #Mocht het programma dingen niet herkennen.. geef in in definition.
    batch_iter = 1
    avg_loss = 0
    correct = 0
    total = 0
    epoch_loss = 0

    # batch_time = AverageMeter('Time', ':6.3f')
    # loss_running = AverageMeter('Loss', ':.4e')
    # acc_running = AverageMeter('Accuracy', ':.3f')

    # progress = ProgressMeter(
    #     len(train_dataset),
    #     [batch_time, loss_running, acc_running],
    #     prefix="Train, epoch: [{}]".format(epoch))

    # start = time.time()

    subsetN = ceil(len(query_train_set) / config['cache_refresh_rate'])
    subsetIdx = np.array_split(np.arange(len(query_train_set)), subsetN)

    n_batches = (len(query_train_set) + config['batch_size'] - 1) // config['batch_size']

    refresh_cache_epoch_step = int(config['cache_refresh_rate'] / config['batch_size'])
    


    train_loader = DataLoader(dataset = train_dataset, batch_size=4) # TODO hier moeten we gewoon de functions gaan maken zoals in pittsburg250k van Nanne

    # COMMENT OUT WHENEVER IT HAS CRASHED AFTER THIS CODE AS THE CACHE IS ALREADY MADE:  
    for subIter in range(subsetN):
        print('Loading all training data into Cache')
        model.eval()
        query_train_set.cache = 'query_train_set_cache' #join(opt.cachePath, train_set.whichSet + '_feat_cache.hdf5')
        with h5py.File(query_train_set.cache, mode='w') as h5:
            pool_size = encoder_dim
            if config['NetVLAD']== True: 
              pool_size *= config['clusters']
            h5feat = h5.create_dataset("features",
                                       [len(train_dataset), pool_size],
                                       dtype=np.float32)
            with torch.no_grad():
                for iteration, (inputs, indices) in enumerate(train_loader, 1):
                    inputs = inputs.to(device)
                    #labels = labels.to(device)
                    #model = model.to(device)
                    imageCNN = model.forwardCNN(inputs)
                    pooling = model.pooling(imageCNN)
                    h5feat[indices.detach().numpy(), :] = pooling.detach().cpu().numpy()
                    del inputs, imageCNN, pooling

        sub_train_set = Subset(dataset=query_train_set, indices=subsetIdx[subIter])

        training_data_loader = DataLoader(dataset=sub_train_set, num_workers=config['num_workers'],
                                          batch_size=config['batch_size'], shuffle=True,
                                          collate_fn=collate_fn, pin_memory=config['cuda'])

        model.train()
        for i, (query, positives, negatives, negCounts, indices) in enumerate(training_data_loader, batch_iter):
            inputs = torch.cat([query, positives, negatives])

            print('Another query speciaal voor jou ==>')

            optimizer.zero_grad()

            inputs = inputs.to(device)
            imageCNN = model.forwardCNN(inputs)
            pooling = model.pooling(imageCNN)

            B, C, H, W = query.shape
            totalnegatives = torch.sum(negCounts)
            vladAnchor, vladPos, vladNeg = torch.split(pooling, [B, B, totalnegatives])

            loss = 0
            # allows for varying length of negatives list.
            for i, negCount in enumerate(negCounts):
                for k in range(negCount):
                    negIx = (torch.sum(negCounts[:i]) + k).item()
                    criterion = nn.TripletMarginLoss(margin=config['margin'] ** 0.5, p=2, reduction='sum')
                    loss += criterion(vladAnchor[i:i + 1], vladPos[i:i + 1], vladNeg[negIx:negIx + 1])

            loss = loss / totalnegatives
            print(loss)
            loss.backward()
            optimizer.step()

            print('Allocated:', torch.cuda.memory_allocated())
            print('Cached:', torch.cuda.memory_cached())

            # now, before going into the next batch, you need to
            # 1) delete all lists corresponding to this batch
            # 2) add the loss of this batch to the total epoch loss

            del inputs, imageCNN, pooling, vladAnchor, vladPos, vladNeg
            del query, positives, negatives

            # should I add a loss.item thing here?
            epoch_loss += loss
            loss = 0  # reset loss

            # progress.display(epoch_step)

            # Measure time
            # batch_time.update(time.time() - start)
            # start = time.time()

            # if epoch_step == refresh_cache_epoch_step:
            #     negCache = None

        batch_iter += len(training_data_loader)  # train_loader moet dus lengte van 4 (batchsize) hebben
        optimizer.zero_grad()
        del training_data_loader
        torch.cuda.empty_cache()
        remove(query_train_set.cache)
    avg_loss = epoch_loss / n_batches

    return avg_loss

def test(validation_dataset, epoch=0, write_tboard=False):
  #use validation_dataset as defined in def main
  model.eval()
  test_data_loader = DataLoader(dataset=validation_dataset, 
                num_workers=config['num_workers'], batch_size=config['batch_size'], shuffle=False, 
                pin_memory=config['cuda'])
  #Disabling gradient calculation is useful for inference, when you are sure that you will not call Tensor.backward(). It will reduce memory consumption for computations that would otherwise have requires_grad=True.
  with torch.no_grad():
    pool_size = encoder_dim
    if config['NetVLAD'] == True:
      pool_size *= config['clusters']
    dbFeat = np.empty((len(validation_dataset), pool_size))
    
    for iteration, (input, indices) in enumerate(test_data_loader, 1):
      # inputs = input.to(device)
      imageCNN = model.forwardCNN(inputs)
      pooling = model.pooling(imageCNN)
    dbFeat[indices.detach().numpy(), :] = pooling.detach().cpu().numpy()
    
    #Show proceedings
    # if iteration % 50 == 0 or len(test_data_loader) <= 10:
    #             print("==> Batch ({}/{})".format(iteration, 
    #                 len(test_data_loader)), flush=True)
                
    del input, imageCNN, pooling
  del test_data_loader

  qFeat = dbFeat[validation_dataset.dbStruct.numDb:].astype('float32')
  dbFeat = dbFeat[:validation_dataset.dbStruct.numDb].astype('float32')

  print('====> Building faiss index')
  faiss_index = faiss.IndexFlatL2(pool_size)
  faiss_index.add(dbFeat)

  print('Remember: calculating Recall @ only N=1 right now.')
  print('Here we go.')
  n_values = [5] #vul deze lijst aan als je meer N wil a mattie

  _, predictions = faiss_index.search(qFeat, max(n_values))
  positives = validation_dataset.getPositivesForEval() 

  correct_at_n = np.zeros(len(n_values))
    #TODO can we do this on the matrix in one go?
  for qIx, pred in enumerate(predictions):
      for i,n in enumerate(n_values):
          # if in top N then also in top NN, where NN > N
          if np.any(np.in1d(pred[:n], gt[qIx])):
              correct_at_n[i:] += 1
              break

  recall_at_n = correct_at_n / validation_dataset.dbStruct.numQ

  recalls = {} #make dict for output
  for i,n in enumerate(n_values):
      recalls[n] = recall_at_n[i]
      print("====> Recall@{}: {:.4f}".format(n, recall_at_n[i]))
      if write_tboard: writer.add_scalar('Val/Recall@' + str(n), recall_at_n[i], epoch)
    


from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torchvision.transforms as transforms
import torch.utils.data as data

from os.path import join, exists
from scipy.io import loadmat
import numpy as np
from random import randint, random
from collections import namedtuple
from PIL import Image

from sklearn.neighbors import NearestNeighbors
import h5py

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()

root_dir = r'gdrive/MyDrive/Datafiles_mat/'
root_dir_images = r'gdrive/MyDrive/Datasets/'

dbStruct = namedtuple('dbStruct', ['whichSet', 'dataset',
    'dbImage', 'utmDb', 'qImage', 'utmQ', 'numDb', 'numQ',
    'posDistThr', 'posDistSqThr', 'nonTrivPosDistSqThr'])

def parse_database(path):
  mat = loadmat(path)
  matStruct = mat['dbStruct'].item()

  whichSet = matStruct[0].item()
  dataset = 'pittsburgh'
  dbImage = [f[0].item() for f in matStruct[1]]
  utmDb = matStruct[2].T

  qImage = [f[0].item() for f in matStruct[3]]
  utmQ = matStruct[4].T

  numDb = matStruct[5].item()
  numQ = matStruct[6].item()

  posDistThr = matStruct[7].item()
  posDistSqThr = matStruct[8].item()
  nonTrivPosDistSqThr = matStruct[9].item()

  # dbImage = dbImage[:len(dbImage)//2]
  # utmDb = utmDb[:len(utmDb)//2]
  # qImage = qImage[:len(qImage)//2]
  # utmQ = dbImage[:len(utmQ)//2]
  # numDb = numDb//2
  # numQ = numQ//2


  return dbStruct(whichSet, dataset, dbImage, utmDb, qImage,
          utmQ, numDb, numQ, posDistThr,
          posDistSqThr, nonTrivPosDistSqThr)


class DatasetImages(Dataset):
    def __init__(self, structFile, input_transform=True):
        super().__init__()

        self.input_transform = input_transform
        self.dbStruct = parse_database(structFile)
        self.dataset = self.dbStruct.dataset
        self.images = [dbIm for dbIm in self.dbStruct.dbImage]

        self.positives = None
        self.distances = None

    def __getitem__(self, index):
        image = Image.open(self.images[index])
        # t = self.input_transform()
        # image = t(image)
        image = self.input_transform(image)
        #print('tensor',image)

        return image, index

    def __len__(self):
        return len(self.images)

    def getPositivesForEval(self):
        knn = NearestNeighbors(n_jobs=-1)
        knn.fit(self.dbStruct.utmDb)

        self.distances, self.pot_positives_eval = knn.radius_neighbors(self.dbStruct.utmQ,
                                                                       radius=self.dbStruct.posDistThr)

        return self.pot_positives_eval


class QueryImages(Dataset):
    def __init__(self, structFile, nNegSample=1000, nNeg=10, nPos=10, margin=0.1, input_transform=True):
        super().__init__()

        self.input_transform = input_transform
        self.dbStruct = parse_database(structFile)
        self.cache = 'query_train_set_cache'
        self.margin = margin
        self.nNegSample = nNegSample  # number of negatives to randomly sample
        self.nNeg = nNeg  # number of negatives used for training

        self.images = [dbIm for dbIm in self.dbStruct.dbImage]

        knn = NearestNeighbors(n_jobs=-1)
        knn.fit(self.dbStruct.utmDb)

        self.distances, self.pot_positives = knn.radius_neighbors(self.dbStruct.utmQ,
                                                                  radius=self.dbStruct.nonTrivPosDistSqThr ** 0.5)

        for i, posi in enumerate(self.pot_positives):
            self.pot_positives[i] = np.sort(posi)

        self.hard_negatives = []
        self.distancesforNeg, self.positivesforNeg = knn.radius_neighbors(self.dbStruct.utmQ,
                                                                          radius=self.dbStruct.posDistThr)
        for i in self.positivesforNeg:
            self.hard_negatives.append(np.setdiff1d(np.arange(self.dbStruct.numDb), i, assume_unique=True))

        self.queries = np.where(np.array([len(x) for x in self.pot_positives]) > 0)[0]

        self.negCache = [np.empty((0,)) for _ in range(self.dbStruct.numQ)]

    def __getitem__(self, index):
        index = self.queries[index]  # re-map index to match dataset
        with h5py.File(self.cache, mode='r') as h5:
            h5feat = h5.get("features")

            #qOffset = self.dbStruct.numDb
            qFeat = h5feat[index]

            posFeat = h5feat[self.pot_positives[index].tolist()]
            knn = NearestNeighbors(n_jobs=-1)  # TODO replace with faiss?
            knn.fit(posFeat)
            dPos, posNN = knn.kneighbors(qFeat.reshape(1, -1), 1)
            dPos = dPos.item()
            posIndex = self.pot_positives[index][posNN[0]].item()

            negSample = np.random.choice(self.hard_negatives[index], self.nNegSample)
            negSample = np.unique(np.concatenate([self.negCache[index], negSample]))

            negFeat = h5feat[negSample.tolist()]
            knn.fit(negFeat)

            dNeg, negNN = knn.kneighbors(qFeat.reshape(1, -1),
                                         self.nNeg * 10)  # to quote netvlad paper code: 10x is hacky but fine
            dNeg = dNeg.reshape(-1)
            negNN = negNN.reshape(-1)

            # try to find negatives that are within margin, if there aren't any return none
            violatingNeg = dNeg < dPos + self.margin ** 0.5

            if np.sum(violatingNeg) < 1:
                # if none are violating then skip this query
                return None

            negNN = negNN[violatingNeg][:self.nNeg]
            negIndices = negSample[negNN].astype(np.int32)
            self.negCache[index] = negIndices

        query = Image.open(self.dbStruct.qImage[index])
        positive = Image.open(self.dbStruct.dbImage[posIndex])


        query = self.input_transform(query)
        positive = self.input_transform(positive)

        negatives = []
        for negIndex in negIndices:
            negative = Image.open(self.dbStruct.dbImage[negIndex])
            #if self.input_transform:
            negative = self.input_transform(negative)
            negatives.append(negative)

        negatives = torch.stack(negatives, 0)

        return query, positive, negatives, [index, posIndex] + negIndices.tolist()

    def __len__(self):
        return len(self.queries)


def input_transform():
    return transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize(100),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                               std=[0.229, 0.224, 0.225]),
    ])

def get_training_dataset():
  file_dir = join(root_dir, 'pitts250k_train.mat')
  return DatasetImages(file_dir, input_transform=input_transform())

def get_val_dataset():
  file_dir = join(root_dir, 'pitts250k_val.mat')
  return DatasetImages(file_dir, input_transform=input_transform())

def get_test_dataset():
  file_dir = join(root_dir, 'pitts250k_test.mat')
  return DatasetImages(file_dir, input_transform=input_transform())

def get_training_query_dataset():
  file_dir = join(root_dir, 'pitts250k_train.mat')
  return QueryImages(file_dir, input_transform=input_transform())

def get_val_query_dataset():
  file_dir = join(root_dir, 'pitts250k_val.mat')
  return QueryImages(file_dir, input_transform=input_transform())

def get_test_query_dataset():
  file_dir = join(root_dir, 'pitts250k_test.mat')
  return QueryImages(file_dir, input_transform=input_transform())

def collate_fn(batch):
  batch = list(filter (lambda x:x is not None, batch))
  if len(batch) == 0: 
    return None, None, None, None, None

  query, positive, negatives, indices = zip(*batch)

  query = data.dataloader.default_collate(query)
  positive = data.dataloader.default_collate(positive)
  negCounts = data.dataloader.default_collate([x.shape[0] for x in negatives])
  negatives = torch.cat(negatives, 0)
  import itertools
  indices = list(itertools.chain(*indices))

  return query, positive, negatives, negCounts, indices

import torch
import torch.nn as nn
#from .utils import load_state_dict_from_url
from typing import Any
import torch.nn.functional as F


class NetVLAD(nn.Module):
    '''Only need NetVLAD class.
    CNN layers were imported'''

    def __init__(self, num_clusters=64, dim=128,
                 normalize_input=True):
        super(NetVLAD, self).__init__()
        self.num_clusters = num_clusters
        self.dim = dim
        self.alpha = 0
        self.normalize_input = normalize_input
        self.conv = nn.Conv2d(dim, num_clusters, kernel_size=(1, 1), bias=True)
        self.centroids = nn.Parameter(torch.rand(num_clusters, dim))
        # self._init_params() Waar hebben we dit voor nodig?

    def init_params(self, clsts, traindescs):
        knn = NearestNeighbors(n_jobs=-1)
        knn.fit(traindescs)
        del traindescs
        dsSq = np.square(knn.kneighbors(clsts, 2)[1])
        del knn
        self.alpha = (-np.log(0.01) / np.mean(dsSq[:, 1] - dsSq[:, 0])).item()
        self.centroids = nn.Parameter(torch.from_numpy(clsts))
        del clsts, dsSq

        self.conv.weight = nn.Parameter(
            (2.0 * self.alpha * self.centroids).unsqueeze(-1).unsqueeze(-1)
        )
        self.conv.bias = nn.Parameter(
            - self.alpha * self.centroids.norm(dim=1)
        )

    # based on lyakaap: https://github.com/lyakaap/NetVLAD-pytorch/blob/master/netvlad.py
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        N, C = x.shape[:2]

        x = F.normalize(x, p=2, dim=1)  # design choice: we always normalize before NETVLAD

        # soft-assignment
        soft_assign = self.conv(x).view(N, self.num_clusters, -1)
        soft_assign = F.softmax(soft_assign, dim=1)

        x_flatten = x.view(N, C, -1)

        # calculate residuals to each clusters
        vlad = torch.zeros([N, self.num_clusters, C], dtype=x.dtype, layout=x.layout, device=x.device)
        for C in range(self.num_clusters):  # slower than non-looped, but lower memory usage
            residual = x_flatten.unsqueeze(0).permute(1, 0, 2, 3) - \
                       self.centroids[C:C + 1, :].expand(x_flatten.size(-1), -1, -1).permute(1, 2, 0).unsqueeze(0)
            residual *= soft_assign[:, C:C + 1, :].unsqueeze(2)
            vlad[:, C:C + 1, :] = residual.sum(dim=-1)

        vlad = F.normalize(vlad, p=2, dim=2)  # intra-normalization
        vlad = vlad.view(x.size(0), -1)  # flatten
        vlad = F.normalize(vlad, p=2, dim=1)  # L2 normalize
        x = vlad

        return x

if __name__ == "__main__":
    print('Here we go! Made by Xander & Thijs:')

    # Set the seed
    torch.manual_seed(68)
    np.random.seed(68)

    ##########GPU COMMANDS#################
    device_name = tf.test.gpu_device_name()
    if device_name != '/device:GPU:0':
      raise SystemError('GPU device not found')
    else:
      print('Found GPU at: {}'.format(device_name))

    device = torch.device('cuda' if config['cuda'] else 'cpu')

    if config['cuda']:
        torch.cuda.manual_seed(68)
    ########LOADING DATASET TO Dataloader ##########

    train_dataset = get_training_dataset() # TODO
    test_dataset = get_test_dataset() 
    validation_dataset = get_val_dataset() # TODO

    # define the data loaders
    train_loader = DataLoader(dataset = train_dataset, batch_size=4) # TODO hier moeten we gewoon de functions gaan maken zoals in pittsburg250k van Nanne

    test_loader = DataLoader(dataset = test_dataset, batch_size=4)#TODO

    val_loader = DataLoader(dataset = validation_dataset, batch_size=4) # TODO

    ###########Utilize DATA######################
    if config['mode'].lower() == 'train':
      print('======> Creating query training set')     
      query_train_set = get_training_query_dataset()
    elif config['mode'].lower() == 'test':
      test_set_positives = FormPositives(test_dataset)   
    elif config['mode'].lower() == 'cluster':
      print('dataset already loaded')  
    else:
      raise ValueError('Choose an existing mode (lowercase)')

    
    print('We loaded all data. ')
    print('Building Model ===> ')

    #compared to Nanne, our model WILL be fully pretrained as in the paper.
    #so we cannot use the YourModel implementation we built earlier 

    ################BUILDING THE MODEL WITH GIVEN CONFIGURATION################
    ######since we need pretrained, netvlad class now only contains netvlad####
    ######and we load in the CNN structures before this.#######################
    if config['CNN'].lower() == 'vgg':
      print('VGG it isssss ===>>>')
      encoder_dim = 512

      forwardCNN = models.vgg16(pretrained = True)
      layers = list(forwardCNN.features.children())[:-2] #don't use the linear layers 
      for l in layers[:-5]: 
        for p in l.parameters():
          p.requires_grad = False

    if config['CNN'].lower() == 'alexnet':
      encoder_dim = 256

      forwardCNN = models.alexnet(pretrained = True)
      layers = list(forwardCNN.features.children())[:-2] #don't use the linear layers 
      for l in layers[:-1]:
                for p in l.parameters():
                    p.requires_grad = False


    if config['mode'] == 'cluster':
      layers.append(L2Norm())


    forwardCNN = nn.Sequential(*layers)
    model = nn.Module() 
    model.add_module('forwardCNN', forwardCNN)
    model = model.to(device)

    #for clustering, never use netvlad, 
    #for training and testing, depends if you turned it on

    if config['mode'].lower() == 'train' or config['mode'].lower() == 'test':
      if config['NetVLAD'] == True:
        net_vlad = NetVLAD(num_clusters = config['clusters'], dim=encoder_dim)
        #no experiences with caching, so this we took from Nanne
        if config['mode'] == 'train':
            # initcache = join(config['myPath'] , 'centroids', config['CNN'] + '_' + 'query_train_set' + '_' + str(config['clusters']) +'_desc_cen.hdf5')
            initcache = 'centroids_trainingset'
        else:
            initcache = join(config['myPath'] , 'centroids', config['CNN'] + '_' + 'test_dataset' + '_' + str(config['clusters']) +'_desc_cen.hdf5')

        # if not exists(initcache):
        #     raise FileNotFoundError('Could not find clusters, please run with --mode=cluster before proceeding')

        with h5py.File(initcache, mode='r') as h5: 
            clsts = h5.get("centroids")[...]
            traindescs = h5.get("descriptors")[...]
            net_vlad.init_params(clsts, traindescs) 
            del clsts, traindescs

            model.add_module('pooling', net_vlad)

      elif config['NetVLAD'] == False:
          global_pool = nn.AdaptiveMaxPool2d((1,1))
          model.add_module('pooling', nn.Sequential(*[global_pool, Flatten(), L2Norm()]))   
      else:
          raise ValueError('Yo Bro, NetVLAD should be True/False')


      
      model.forwardCNN = nn.DataParallel(model.forwardCNN)
      if config['mode'] != 'cluster':
              model.pooling = nn.DataParallel(model.pooling)
      isParallel = True
      # model = model.to(device)
      #####BUILD OPTIMIZER#########################
      optimizer = optim.SGD(model.parameters(), lr=config['learning_rate'], momentum=config['momentum'], weight_decay = config['weightdecay'])
      #How to set arguments of optimizer() when loading a pretrained net with its gradient fix to a new net.?^^
      scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=config['lrStep'], gamma=config['lrGamma'])


      criterion = nn.TripletMarginLoss(margin=config['margin']**0.5, 
                p=2, reduction='sum').to(device)



      #####################################################################
      ####PERFORM ACTUAL TEST/TRAINING/CLUSTERING##########################

    if config['mode'] == 'test':
      print('===> Running evaluation step')
      epoch = 1
      recalls = test(test_set, epoch, write_tboard=False)
    elif config['mode'] == 'cluster':
      print('===> Calculating descriptors and clusters')
      centroids(train_dataset)
    elif config['mode'] == 'train':
      print('===> Training model')
      if config['NetVLAD'] == True:
        pooling = 'NetVLAD'
      else: 
        pooling = 'maxpooling'
      writer = SummaryWriter(log_dir=join(config['myPath'] , datetime.now().strftime('%b%d_%H-%M-%S')+'_'+ config['CNN'] +'_'+ pooling))

      not_improved = 0
      best_score = 0
      for epoch in range(1, config['epochs']  + 1):
          scheduler.step(epoch)
          train_epoch(epoch, train_dataset, query_train_set, model)
      
          recalls = test(test_dataset, epoch, write_tboard=True)
          is_best = recalls[5] > best_score 
          if is_best:
              not_improved = 0
              best_score = recalls[5]
          else: 
              not_improved += 1

          if config['patience_earlystopping']  > 0 and not_improved > (config['patience_earlystopping'] ):
              print('Performance did not improve for', config['patience_earlystopping'], 'epochs. Time to (Early)Stop.')
              break

      print("=> Best Recall@5: {:.4f}".format(best_score), flush=True)
      writer.close()


Here we go! Made by Xander & Thijs:
Found GPU at: /device:GPU:0
======> Creating query training set
We loaded all data. 
Building Model ===> 
VGG it isssss ===>>>
===> Training model
Loading all training data into Cache
Another query speciaal voor jou ==>
tensor(0.3285, device='cuda:0', grad_fn=<DivBackward0>)
Allocated: 136396288
Cached: 1153433600
Another query speciaal voor jou ==>
tensor(0.2893, device='cuda:0', grad_fn=<DivBackward0>)
Allocated: 137124352
Cached: 1153433600
Another query speciaal voor jou ==>
tensor(0.3064, device='cuda:0', grad_fn=<DivBackward0>)
Allocated: 137124352
Cached: 1153433600
Another query speciaal voor jou ==>
tensor(0.3159, device='cuda:0', grad_fn=<DivBackward0>)
Allocated: 137124352
Cached: 1153433600
Another query speciaal voor jou ==>
tensor(0.3149, device='cuda:0', grad_fn=<DivBackward0>)
Allocated: 137124352
Cached: 1153433600
Another query speciaal voor jou ==>
tensor(0.3067, device='cuda:0', grad_fn=<DivBackward0>)
Allocated: 137124352
Cached: